Se importan las librerías necesarias para manejo de APIs, scraping, base de datos y variables de entorno.

In [4]:
import os
import time
import requests
import sqlite3
from bs4 import BeautifulSoup
from dotenv import load_dotenv


ModuleNotFoundError: No module named 'requests'

Se carga la API key desde un archivo .env y se configuran URLs base y cabeceras para scraping.

In [ ]:
# Cargar variables de entorno
load_dotenv()
API_KEY = os.getenv("GOOGLE_BOOKS_API_KEY")

# Constantes
BASE_URL = "http://books.toscrape.com/"
CATALOGUE_URL = BASE_URL + "catalogue/"
headers = {"User-Agent": "Mozilla/5.0"}
cache_autores = {}


Se conecta a la base de datos SQLite y se crean las tablas necesarias para almacenar libros, autores y géneros.

In [ ]:
conn = sqlite3.connect("libros.db")
cursor = conn.cursor()

cursor.executescript("""
CREATE TABLE IF NOT EXISTS autores (
    id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    nombre TEXT UNIQUE NOT NULL
);

CREATE TABLE IF NOT EXISTS generos (
    id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    nombre TEXT UNIQUE NOT NULL
);

CREATE TABLE IF NOT EXISTS libros (
    id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    titulo TEXT NOT NULL,
    precio TEXT NOT NULL,
    stock TEXT NOT NULL,
    url TEXT NOT NULL,
    rating INTEGER NOT NULL,
    genero_id INTEGER NOT NULL,
    FOREIGN KEY (genero_id) REFERENCES generos(id)
);

CREATE TABLE IF NOT EXISTS autor_libro (
    autor_id INTEGER NOT NULL,
    libro_id INTEGER NOT NULL,
    PRIMARY KEY (autor_id, libro_id),
    FOREIGN KEY (autor_id) REFERENCES autores(id),
    FOREIGN KEY (libro_id) REFERENCES libros(id)
);
""")


Consulta a la API de Google Books para obtener el autor de un libro a partir del título.

In [ ]:
def buscar_autor_google_books(titulo):
    if titulo in cache_autores:
        return cache_autores[titulo]

    url = "https://www.googleapis.com/books/v1/volumes"
    params = {"q": f"intitle:{titulo}", "key": API_KEY, "maxResults": 1}

    try:
        response = requests.get(url, params=params, timeout=5)
        response.raise_for_status()
        data = response.json()
        if "items" in data:
            volumen = data["items"][0]["volumeInfo"]
            autores = volumen.get("authors", ["Desconocido"])
            autor = ", ".join(autores)
        else:
            autor = "No encontrado"
    except Exception:
        autor = "Error API"

    cache_autores[titulo] = autor
    time.sleep(1)
    return autor


Obtiene género y stock de un libro haciendo scraping del sitio web.



In [5]:
def obtener_detalle_libro(url_relativa):
    url_libro = CATALOGUE_URL + url_relativa.lstrip("./")
    try:
        detalle = requests.get(url_libro, headers=headers, timeout=5)
        detalle.raise_for_status()
        soup = BeautifulSoup(detalle.text, "html.parser")

        breadcrumb = soup.select("ul.breadcrumb li a")
        genero = breadcrumb[2].text.strip() if len(breadcrumb) >= 3 else "Desconocido"

        tabla = soup.find("table", class_="table table-striped")
        stock = (
            tabla.find_all("tr")[5].find("td").text.strip() if tabla else "Sin stock"
        )
    except Exception:
        genero = "Desconocido"
        stock = "Sin stock"

    return genero, stock, url_libro


Insertan libros, autores y géneros en la base de datos, evitando duplicados.

In [6]:
def obtener_o_insertar_id(nombre, tabla):
    cursor.execute(f"SELECT id FROM {tabla} WHERE nombre = ?", (nombre,))
    resultado = cursor.fetchone()
    if resultado:
        return resultado[0]
    cursor.execute(f"INSERT INTO {tabla} (nombre) VALUES (?)", (nombre,))
    return cursor.lastrowid

def insertar_libro(autor_str, titulo, precio, genero_str, stock, url, rating):
    cursor.execute("SELECT id FROM libros WHERE titulo = ? AND url = ?", (titulo, url))
    if cursor.fetchone():
        print(f"🔁 Libro duplicado ignorado: {titulo}")
        return

    genero_id = obtener_o_insertar_id(genero_str, "generos")

    cursor.execute("""
        INSERT INTO libros (titulo, precio, stock, url, rating, genero_id)
        VALUES (?, ?, ?, ?, ?, ?)
    """, (titulo, precio, stock, url, rating, genero_id))
    libro_id = cursor.lastrowid

    for nombre_autor in [a.strip() for a in autor_str.split(",")]:
        autor_id = obtener_o_insertar_id(nombre_autor, "autores")
        cursor.execute("""
            INSERT OR IGNORE INTO autor_libro (autor_id, libro_id)
            VALUES (?, ?)
        """, (autor_id, libro_id))

    conn.commit()
    print(f"✅ Insertado: {titulo}")



Extraen categorías y libros por página haciendo scraping paginado.



In [7]:
RATING_MAP = {"One": 1, "Two": 2, "Three": 3, "Four": 4, "Five": 5}

def obtener_categorias():
    categorias = {}
    try:
        response = requests.get(BASE_URL, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")
        links = soup.select("div.side_categories ul li ul li a")
        for link in links:
            nombre = link.text.strip()
            href = link.get("href").replace("index.html", "")
            categorias[nombre] = BASE_URL + href
    except Exception as e:
        print("⚠️ Error al obtener categorías:", e)
    return categorias

def obtener_libros_de_categoria(nombre_categoria, url_categoria):
    libros = []
    pagina = 1

    while True:
        url_pagina = (
            url_categoria + f"page-{pagina}.html"
            if pagina > 1
            else url_categoria + "index.html"
        )
        try:
            response = requests.get(url_pagina, headers=headers, timeout=10)
            if response.status_code == 404:
                break
            response.raise_for_status()
        except Exception as e:
            print(f"[!] Error en {nombre_categoria}, página {pagina}: {e}")
            break

        soup = BeautifulSoup(response.text, "html.parser")
        articulos = soup.find_all("article", class_="product_pod")
        if not articulos:
            break

        for libro in articulos:
            rating_texto = libro.find("p", class_="star-rating")["class"][1]
            rating_numero = RATING_MAP.get(rating_texto, 0)
            titulo = libro.h3.a["title"]
            precio = libro.find("p", class_="price_color").text.replace("Â", "")
            url_relativa = libro.h3.a["href"]

            genero, stock, url_completo = obtener_detalle_libro(url_relativa)
            autor = buscar_autor_google_books(titulo)

            libros.append(
                (autor, titulo, precio, genero, stock, url_completo, rating_numero)
            )

        pagina += 1

    return libros


Se ejecuta el scraping de todas las categorías y se insertan los libros en la base de datos.

In [8]:
categorias = obtener_categorias()

for nombre, url_categoria in categorias.items():
    print(f"\n📚 Categoría: {nombre}")
    libros = obtener_libros_de_categoria(nombre, url_categoria)
    for libro in libros:
        insertar_libro(*libro)

print("\n✅ Todos los libros insertados correctamente.")


⚠️ Error al obtener categorías: name 'requests' is not defined

✅ Todos los libros insertados correctamente.


Exporta los resultados filtrados a un archivo CSV.

In [9]:
import csv

def exportar_a_csv(libros):
    nombre_archivo = "libros_filtrados.csv"
    with open(nombre_archivo, mode="w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["Título", "Precio", "Stock", "Rating", "URL", "Género", "Autor(es)"])
        for libro in libros:
            writer.writerow(libro)
    print(f"✅ Resultados exportados correctamente a '{os.path.abspath(nombre_archivo)}'")


In [ ]:
def conectar_db():
    return sqlite3.connect("libros.db")


def mostrar_menu():
    print("\n=== FILTROS DISPONIBLES ===")
    print("1. Filtrar por rating")
    print("2. Filtrar por precio máximo")
    print("3. Filtrar por género")
    print("4. Filtrar por autor")
    print("5. Filtrar por disponibilidad")
    print("6. Ver todos los libros")
    print("0. Salir")


def construir_query(opcion):
    query_base = """
    SELECT libros.titulo, libros.precio, libros.stock, libros.rating,
           libros.url, generos.nombre AS genero, GROUP_CONCAT(autores.nombre, ', ') AS autores
    FROM libros
    JOIN generos ON libros.genero_id = generos.id
    JOIN autor_libro ON libros.id = autor_libro.libro_id
    JOIN autores ON autores.id = autor_libro.autor_id
    """
    condiciones = []
    parametros = []

    if opcion == "1":
        while True:
            rating = input(" Ingrese rating (1-5): ").strip()
            if rating.isdigit() and 1 <= int(rating) <= 5:
                break
            print("Rating inválido. Debe ser un número del 1 al 5.")
        condiciones.append("libros.rating = ?")
        parametros.append(rating)

    elif opcion == "2":
        while True:
            precio_max = input(" Ingrese precio máximo (sin símbolo): ").strip()
            try:
                float(precio_max)
                break
            except ValueError:
                print("Ingrese un número válido.")
        condiciones.append("CAST(SUBSTR(libros.precio, 2) AS REAL) <= ?")
        parametros.append(precio_max)

    elif opcion == "3":
        genero = input("Ingrese nombre del género: ").strip()
        condiciones.append("generos.nombre LIKE ?")
        parametros.append(f"%{genero}%")

    elif opcion == "4":
        autor = input("Ingrese nombre del autor: ").strip()
        condiciones.append("autores.nombre LIKE ?")
        parametros.append(f"%{autor}%")

    elif opcion == "5":
        condiciones.append("libros.stock NOT LIKE '%(0 available)%'")

    where_clause = f"WHERE {' AND '.join(condiciones)}" if condiciones else ""
    query = f"""
    {query_base}
    {where_clause}
    GROUP BY libros.id
    ORDER BY libros.titulo;
    """
    return query, parametros


def mostrar_resultados(cursor):
    libros = cursor.fetchall()
    if not libros:
        print("\n📭 No se encontraron libros con ese filtro.")
        return

    print(f"\nResultados encontrados: {len(libros)}\n")
    for libro in libros:
        titulo, precio, stock, rating, url, genero, autores = libro
        print(
            f"Título: {titulo}\nPrecio: {precio}\nStock: {stock}\n⭐ Rating: {rating}"
        )
        print(
            f"Género: {genero}\nAutor(es): {autores}\nLink: {url}\n{'-'*40}"
        )

    exportar = (
        input("\n¿Desea exportar estos resultados a CSV? (s/n): ").strip().lower()
    )
    if exportar == "s":
        exportar_a_csv(libros)


def main():
    conn = conectar_db()
    cursor = conn.cursor()

    while True:
        mostrar_menu()
        opcion = input("Seleccione una opción: ").strip()

        if opcion == "0":
            print("👋 Saliendo del visor...")
            break
        elif opcion in {"1", "2", "3", "4", "5"}:
            query, params = construir_query(opcion)
            cursor.execute(query, params)
            mostrar_resultados(cursor)
        elif opcion == "6":
            query, params = construir_query("6")  # sin filtros
            cursor.execute(query)
            mostrar_resultados(cursor)
        else:
            print("❌ Opción no válida. Intente nuevamente.")

    conn.close()


if __name__ == "__main__":
    main()

NameError: name 'sqlite3' is not defined

+------------------+           +----------------------+           +------------------+
|     autores      |           |    autor_libro       |           |     libros       |
+------------------+           +----------------------+           +------------------+
| id (PK)          |<--------->| autor_id (PK, FK)    |<--------->| id (PK)          |
| nombre (UNIQUE)  |           | libro_id (PK, FK)    |           | titulo           |
+------------------+           +----------------------+           | precio           |
                                                           | stock            |
                                                           | url              |
                                                           | rating           |
                                                           | genero_id (FK)   |
                                                           +------------------+
                                                                   |
                                                                   |
                                                           +------------------+
                                                           |     generos      |
                                                           +------------------+
                                                           | id (PK)          |
                                                           | nombre (UNIQUE)  |
                                                           +------------------+


In [11]:
SELECT a.nombre, COUNT(al.libro_id) AS cantidad_libros
FROM autores a
JOIN autor_libro al ON a.id = al.autor_id
GROUP BY a.id
ORDER BY cantidad_libros DESC
LIMIT 5;


SyntaxError: invalid syntax (2691234501.py, line 1)